#**Pré-processamento**

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

import pandas as pd
import seaborn as sns
import missingno as msno
import warnings
from matplotlib import pyplot as plt
import datetime
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

df = pd.read_csv("gdrive/My Drive/dissertação/Qualidade_do_ar_-_Dados_horarios.csv",sep=",")
df['Data']=df['Data'].astype('str')
df['Data']=df['Data'].apply(lambda x: x[:-3])

#conversão para datetime
def convert_to_datetime(x):
    return(datetime.datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))
df['Data']=df['Data'].apply(convert_to_datetime) 
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-8:-3])
# #extrair ano, mes, hora 
df['Ano']=df['Data'].apply(lambda x: x.year)
df['Mês']=df['Data'].apply(lambda x: x.month)
df['Dia']=df['Data'].apply(lambda x: x.day)
df['Hora']=df['Data'].apply(lambda x: x.hour)
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-11:-6])

df_bg=df[df['Estação']=='BG'] #Bangu
df_gc=df[df['Estação']=='CG'] #Campo Grande
df_ca=df[df['Estação']=='CA'] #Centro
df_av=df[df['Estação']=='AV'] #Copacabana
df_ir=df[df['Estação']=='IR'] #Irajá
df_pg=df[df['Estação']=='PG'] #Pedra de Guaratiba
df_sc=df[df['Estação']=='SC'] #São Cristóvão
df_sp=df[df['Estação']=='SP'] #Tijuca

df_sc.set_index('Data',inplace=True)
df_sc.drop(columns=['OBJECTID','CodNum','Estação','Dir_Vento','NO2','HCNM','HCT','CH4','NO','NOx',\
                   'PM2_5', 'Lat', 'Lon', 'X_UTM_Sirgas2000','Y_UTM_Sirgas2000'], inplace=True)
df_sc_2=df_sc.copy() #para fazer distribuição sem eliminar ausentes

df_sc=df_sc['2011-01-01':'2018-01-01']
#explicar que usou interpolação linear com spline e justificar
df_sc['Chuva'].interpolate(method='slinear', inplace=True)
df_sc['Pres'].interpolate(method='slinear', inplace=True)
df_sc['RS'].interpolate(method='slinear', inplace=True)
df_sc['Temp'].interpolate(method='slinear', inplace=True)
df_sc['UR'].interpolate(method='slinear', inplace=True)
df_sc['Vel_Vento'].interpolate(method='slinear', inplace=True)
df_sc['SO2'].interpolate(method='slinear', inplace=True)
df_sc['CO'].interpolate(method='slinear', inplace=True)
df_sc['O3'].interpolate(method='slinear', inplace=True)
df_sc['PM10'].interpolate(method='slinear', inplace=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#**(spline) LAG=1**

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 1 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 1)
reframed = series_to_supervised(scaled, n_hours, 1) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt)

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 1s - loss: 19.8062 - val_loss: 19.8556
Epoch 2/150
90/90 - 0s - loss: 18.8252 - val_loss: 19.0649
Epoch 3/150
90/90 - 0s - loss: 18.1781 - val_loss: 18.3770
Epoch 4/150
90/90 - 0s - loss: 17.6250 - val_loss: 17.8171
Epoch 5/150
90/90 - 0s - loss: 17.1063 - val_loss: 17.2910
Epoch 6/150
90/90 - 0s - loss: 16.6127 - val_loss: 16.8260
Epoch 7/150
90/90 - 0s - loss: 16.1934 - val_loss: 16.4056
Epoch 8/150
90/90 - 0s - loss: 15.8227 - val_loss: 16.0234
Epoch 9/150
90/90 - 0s - loss: 15.4791 - val_loss: 15.6696
Epoch 10/150
90/90 - 0s - loss: 15.1658 - val_loss: 15.3447
Epoch 11/150
90/90 - 0s - loss: 14.8744 - val_loss: 15.0320
Epoch 12/150
90/90 - 0s - loss: 14.5916 - val_loss: 14.7522
Epoch 13/150
90/90 - 0s - loss: 14.3230 - val_loss: 14.4723
Epoch 14/150
90/90 - 0s - loss: 14.0234 - val_loss: 14.1719
Epoch 15/150
90/90 - 0s - loss: 13.7645 - val_loss: 13.8875
Epoch 16/150
90/90 - 0s - loss: 13.5286 - val_loss: 13.6332
Epoch 17/150
90/90 - 0s - loss: 13.3035 - val_los

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 10.802


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 6.388


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.796


In [ ]:
# plot history
plt.style.use('seaborn')
plt.figure(figsize=(18,9))
pyplot.plot(history.history['loss'], label='train', color='blue')
pyplot.plot(history.history['val_loss'], label='test', color='red')
plt.xlabel('epoch',fontsize=20)
plt.ylabel('loss',fontsize=20) 
# plt.title('train and test loss evolution',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tight_layout()
pyplot.legend(fontsize=20)
pyplot.show()

#**(spline) LAG=2**

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 2 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 2)
reframed = series_to_supervised(scaled, n_hours, 2) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt)

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 1s - loss: 19.6868 - val_loss: 19.4623
Epoch 2/150
90/90 - 0s - loss: 18.4972 - val_loss: 18.6856
Epoch 3/150
90/90 - 0s - loss: 17.8976 - val_loss: 18.1207
Epoch 4/150
90/90 - 0s - loss: 17.4174 - val_loss: 17.6421
Epoch 5/150
90/90 - 0s - loss: 16.9921 - val_loss: 17.2127
Epoch 6/150
90/90 - 0s - loss: 16.5792 - val_loss: 16.7782
Epoch 7/150
90/90 - 0s - loss: 16.1141 - val_loss: 16.2924
Epoch 8/150
90/90 - 0s - loss: 15.7135 - val_loss: 15.8792
Epoch 9/150
90/90 - 0s - loss: 15.3590 - val_loss: 15.5254
Epoch 10/150
90/90 - 0s - loss: 15.0279 - val_loss: 15.1785
Epoch 11/150
90/90 - 0s - loss: 14.6929 - val_loss: 14.8397
Epoch 12/150
90/90 - 0s - loss: 14.3638 - val_loss: 14.5616
Epoch 13/150
90/90 - 0s - loss: 14.0905 - val_loss: 14.2326
Epoch 14/150
90/90 - 0s - loss: 13.8347 - val_loss: 13.9760
Epoch 15/150
90/90 - 0s - loss: 13.5923 - val_loss: 13.7486
Epoch 16/150
90/90 - 0s - loss: 13.3719 - val_loss: 13.5095
Epoch 17/150
90/90 - 0s - loss: 13.1576 - val_los

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 13.805


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 8.743


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.667


#**(spline) LAG=3**

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 3 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 2)
reframed = series_to_supervised(scaled, n_hours, 3) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt)

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 1s - loss: 19.6661 - val_loss: 19.5101
Epoch 2/150
90/90 - 0s - loss: 18.5233 - val_loss: 18.7034
Epoch 3/150
90/90 - 0s - loss: 18.0111 - val_loss: 18.1943
Epoch 4/150
90/90 - 0s - loss: 17.5810 - val_loss: 17.7524
Epoch 5/150
90/90 - 0s - loss: 17.1861 - val_loss: 17.3494
Epoch 6/150
90/90 - 0s - loss: 16.8119 - val_loss: 16.9298
Epoch 7/150
90/90 - 0s - loss: 16.4535 - val_loss: 16.5887
Epoch 8/150
90/90 - 0s - loss: 16.1554 - val_loss: 16.2892
Epoch 9/150
90/90 - 0s - loss: 15.8723 - val_loss: 15.9797
Epoch 10/150
90/90 - 0s - loss: 15.5530 - val_loss: 15.6988
Epoch 11/150
90/90 - 0s - loss: 15.3065 - val_loss: 15.4232
Epoch 12/150
90/90 - 0s - loss: 15.0781 - val_loss: 15.2040
Epoch 13/150
90/90 - 0s - loss: 14.8396 - val_loss: 14.9674
Epoch 14/150
90/90 - 0s - loss: 14.6422 - val_loss: 14.7624
Epoch 15/150
90/90 - 0s - loss: 14.4715 - val_loss: 14.5841
Epoch 16/150
90/90 - 0s - loss: 14.2464 - val_loss: 14.3677
Epoch 17/150
90/90 - 0s - loss: 14.0808 - val_los

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 16.353


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 10.682


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.532


#**(spline) LAG=4**

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 2 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 4)
reframed = series_to_supervised(scaled, n_hours, 4) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt)

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 1s - loss: 19.9510 - val_loss: 19.9674
Epoch 2/150
90/90 - 0s - loss: 18.9248 - val_loss: 19.2017
Epoch 3/150
90/90 - 0s - loss: 18.4616 - val_loss: 18.7127
Epoch 4/150
90/90 - 0s - loss: 18.1449 - val_loss: 18.3815
Epoch 5/150
90/90 - 0s - loss: 17.8476 - val_loss: 18.0902
Epoch 6/150
90/90 - 0s - loss: 17.5781 - val_loss: 17.8029
Epoch 7/150
90/90 - 0s - loss: 17.3306 - val_loss: 17.5413
Epoch 8/150
90/90 - 0s - loss: 17.1146 - val_loss: 17.3179
Epoch 9/150
90/90 - 0s - loss: 16.9224 - val_loss: 17.1119
Epoch 10/150
90/90 - 0s - loss: 16.7281 - val_loss: 16.9203
Epoch 11/150
90/90 - 0s - loss: 16.5328 - val_loss: 16.7121
Epoch 12/150
90/90 - 0s - loss: 16.3443 - val_loss: 16.5403
Epoch 13/150
90/90 - 0s - loss: 16.2035 - val_loss: 16.3351
Epoch 14/150
90/90 - 0s - loss: 16.0221 - val_loss: 16.1915
Epoch 15/150
90/90 - 0s - loss: 15.8450 - val_loss: 16.0102
Epoch 16/150
90/90 - 0s - loss: 15.6870 - val_loss: 15.8515
Epoch 17/150
90/90 - 0s - loss: 15.5865 - val_los

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 19.140


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 12.757


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.359
